In [2]:
import pandas as pd
import numpy as np
import os
from datasets import load_dataset
from dotenv import load_dotenv
import google.generativeai as genai
from minsearch import Index

/home/cirec/Documents/cirec-01-llm-falcon/env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
load_dotenv()
genai.configure(api_key=os.environ["API_KEY"])

In [4]:
# Prepare dataset
# using gotquestions dataset

import json

with open("datasets/questions_answer.json", 'rt') as f_in:
    data = json.load(f_in)

print(data[0])

{'category': 'Questions about God', 'sub.category': 'The Nature of God', 'question': 'What is God like?', 'answer': 'Answer\nEvery culture in the history of the world has had some concept of what God is like. Some have assumed that God is in control of the weather and have made images of a storm god throwing lightning bolts around (Baal worship in Canaan). Some have assumed that God is very powerful, and so they worshiped the most powerful thing they could see, the sun (Ra worship in Egypt). Others have assumed that God is everywhere and therefore have worshiped everything (pantheism in Stoic philosophy). Some have assumed that God is unknowable and have turned to agnosticism or, just to cover their bases, have worshiped “An Unknown God” (Acts 17:23).\nThe problem with each of these assumptions is that they only get part of the picture of who God is. Yes, God is in control of the weather, but He is also in control of so much more. He is powerful, but much more powerful than the sun. He

In [5]:

index = Index(
    text_fields = ["question", "answer", "sub.category"],
    keyword_fields = ["category"]
)

index.fit(data)

In [6]:
def search(query):
# filter_dict = {"category": "Questions about God"}
    boost_dict = {"question": 3, "answer": 1, "sub.category": 1}

    return index.search(query, boost_dict, num_results=5)

In [7]:
def build_prompt(query, search_results):
    prompt_template = """
    You are a spiritual assistant, your goal is to answer questions about christian spirituality like God, Jesus, Holy Spirit, and Life as a believer of this spirituality.
    Answer the QUESTIONS based on the CONTEXT when answering the QUESTION.
    Answer the questions as if you are a spiritual assistant using the CONTEXT given. Don't provide personal opinions or beliefs.
    If the CONTEXT doesn't match or contain the answer, give NONE as the response.
    
    QUESTION : {question}
    
    CONTEXT: {context}
    
    """.strip()

    context = ""
    for doc in search_results:
        context = f'{context}sub.category: {doc["sub.category"]} \nquestion: {doc["question"]} \nanswer: {doc["answer"]}\n\n'

    return prompt_template.format(question= query, context=search_results).strip()

In [8]:
def llm(prompt):
    model = genai.GenerativeModel("gemini-1.5-pro")
    result = model.generate_content(prompt)
    return result.text
    

In [12]:
def rag(query):
    results = search(query)
    print(results)
    prompt = build_prompt(query, results)
    return llm(prompt)

In [15]:
rag("Who is Paul?")

[{'category': 'Questions about People in the Bible', 'sub.category': 'New Testament Individuals', 'question': 'Who was Paul in the Bible?', 'answer': 'Answer\r\nThere is much we can learn from the life of the apostle Paul. Far from ordinary, Paul was given the opportunity to do extraordinary things for the kingdom of God. The story of Paul is a story of redemption in Jesus Christ and a testimony that no one is beyond the saving grace of the Lord. However, to gain the full measure of the man, we must examine his dark side and what he symbolized before becoming “the Apostle of Grace.” Paul’s early life was marked by religious zeal, brutal violence, and the relentless persecution of the early church. Fortunately, the later years of Paul’s life show a marked difference as he lived his life for Christ and for the advancement of His kingdom. \n\r\nPaul was actually born as Saul. He was born in Tarsus in Cilicia, a province in the southeastern corner of modern-day Tersous, Turkey, sometime in

'Answer\nThere is much we can learn from the life of the apostle Paul.  He was born as Saul of Tarsus, a Benjamite Hebrew and Roman citizen raised as a Pharisee. Educated by Gamaliel, he became a zealous lawyer known for persecuting early Christians.  His life changed dramatically on the road to Damascus when he encountered the risen Jesus Christ.  Blinded by the light, he was led to Damascus where Ananias restored his sight and he received the Holy Spirit. From then on, Saul, now called Paul, dedicated his life to spreading the gospel.  He embarked on missionary journeys, wrote many New Testament epistles, and ultimately died as a martyr in Rome.  His story demonstrates that anyone can be saved, become a powerful witness for Christ, and surrender fully to God.\n'

In [ ]:
## Implementing Vector Search using elastic search

from elasticsearch import Elasticsearch
elastic_index = Elasticsearch.index(
    {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}
)